In [1]:
%run init.ipynb

Using TensorFlow backend.


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.CDSSMPreprocessor(fixed_length_left=10, fixed_length_right=10)
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 2118/2118 [00:00<00:00, 5365.33it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 18841/18841 [00:05<00:00, 3205.80it/s]
Building Vocabulary from a datapack.: 100%|██████████| 1614998/1614998 [00:00<00:00, 3031577.24it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8384.04it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3939.48it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 96621.02it/s]
Processing text_left with chain_transform of NgramLetter => WordHashing: 100%|██████████| 2118/2118 [00:07<00:00, 269.84it/s]
Processing text_right with chain_t

In [3]:
model = mz.models.CDSSM()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['filters'] = 64
model.params['kernel_size'] = 3
model.params['strides'] = 1
model.params['padding'] = 'same'
model.params['conv_activation_func'] = 'tanh'
model.params['w_initializer'] = 'glorot_normal'
model.params['b_initializer'] = 'zeros'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 64
model.params['mlp_num_fan_out'] = 64
model.params['mlp_activation_func'] = 'tanh'
model.params['dropout_rate'] = 0.8
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10, 9644)     0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 10, 9644)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 10, 64)       1851712     text_left[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 10, 64)       1851712     text_right[0][0]                 
__________________________________________________________________________________________________
dropout_1 

In [4]:
pred_x, pred_y = test_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20
)
print('num batches:', len(train_generator))

num batches: 102


In [5]:
history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=1, use_multiprocessing=False)

Epoch 1/20
102/102 [==============================] - 65s 635ms/step - loss: 0.8021
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.42304382290227854 - normalized_discounted_cumulative_gain@5(0.0): 0.49915948768338086 - mean_average_precision(0.0): 0.46037758752542035
Epoch 2/20
102/102 [==============================] - 45s 445ms/step - loss: 0.5966
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.43781763271520285 - normalized_discounted_cumulative_gain@5(0.0): 0.520097599085372 - mean_average_precision(0.0): 0.4762598411822459
Epoch 3/20
102/102 [==============================] - 46s 447ms/step - loss: 0.4992
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.44923101748788413 - normalized_discounted_cumulative_gain@5(0.0): 0.5136672947113214 - mean_average_precision(0.0): 0.4803110559647868
Epoch 4/20
102/102 [==============================] - 46s 446ms/step - loss: 0.4143
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.467714954371615 - 